In [ ]:
!pip install rapidfuzz

In [ ]:
# copy & edit from : https://www.kaggle.com/thedrcat/wiki-image-caption-eda-and-baseline-new-data
import pandas as pd
import numpy as np
import base64
from PIL import Image
import io
from rapidfuzz import process, fuzz
from urllib.parse import unquote

## Take a look on submission 

In [ ]:
sub = pd.read_csv('../input/wikipedia-image-caption/sample_submission.csv')
sub.head(15)

#### There are id, caption_title_and_reference_description, in sample submission  
#### and every id has 5 caption

#### test.tsv contain id, image_url

In [ ]:
test = pd.read_csv('../input/wikipedia-image-caption/test.tsv',sep='\t')
test.head()

In [ ]:
for i in range(5):
    print(test.image_url.iloc[i])

In [ ]:
test_caption_list = pd.read_csv('../input/wikipedia-image-caption/test_caption_list.csv')
test_caption_list.head()

In [ ]:
test_image = pd.read_csv('../input/wikipedia-image-caption/image_data_test/image_pixels/test_image_pixels_part-00000.csv',sep='\t',header=None)
test_image.head()

In [ ]:
image64_encode = base64.b64decode(test_image[1].iloc[2])
img = Image.open(io.BytesIO(image64_encode))
img

In [ ]:
import datatable as dt
train = dt.fread('../input/wikipedia-image-caption/train-00000-of-00005.tsv')
train.head()

#### The page_url have contain some information for caption_title_and_reference_description  
#### test data also have url. Maybe it can do something.

In [ ]:
def url_target_process(s):
    s = s.rsplit('/',1)[1]
    s = unquote(s)
    s = s.replace('_',' ')
    s = s + '[SEP]'
    return s

#### from url to target in training data  

In [ ]:
for i in range(5):
    print(f'target: {train[i,-1]}')
    print(f'prediction: {url_target_process(train[i,1])}')
    print()

#### Seems work so apply on test data on make submission

In [ ]:
test['prediction'] = test['image_url'].apply(url_target_process)
test.head()

In [ ]:
Caption = test_caption_list.caption_title_and_reference_description.values.tolist()

In [ ]:
for i in range(5):
    pp = test.prediction[i]
    print(pp)
    res = process.extract(pp, Caption, scorer=fuzz.ratio, processor=None, limit=4)
    print('closet caption:')
    for j in res:
        print(j[0])
    print('*'*40)

In [ ]:
def predict_caption(m):
    res = process.extract(m, Caption, scorer=fuzz.ratio, processor=None, limit=5)
    res = [x[0] for x in res]
    return res

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
test['caption_title_and_reference_description'] = test['prediction'].progress_apply(predict_caption)

In [ ]:
del sub

In [ ]:

sub = test[['id','caption_title_and_reference_description']]
sub = sub.explode('caption_title_and_reference_description')
sub.head()

In [ ]:
sub.to_csv('submission.csv',index=False)